# Comparaison de trois approches d’analyse de tweets et démarche MLOps : Du modèle simple à BERT

Dans cet article, nous allons explorer trois approches différentes pour analyser la polarité (sentiment) de tweets, puis décrire la démarche MLOps mise en œuvre pour automatiser, versionner et déployer ces modèles. Les trois méthodes dont nous parlerons sont :

- Un modèle sur mesure simple : Régression Logistique avec vecteurs TF-IDF.
- Un modèle sur mesure avancé : Réseau de neurones LSTM bidirectionnel (BiLSTM) utilisant des embeddings GloVe.
- Un modèle avancé pré-entraîné : DistilBERT (via la librairie ktrain).

Au fil de cet article, nous présenterons leurs spécificités, leur mise en œuvre et les éléments de comparaison. Nous détaillerons également la mise en place d’une démarche orientée MLOps (Machine Learning Operations), incluant le tracking expérimental, la gestion des versions, les tests, le déploiement, ainsi que le suivi de la performance en production avec un exemple d’utilisation d’Azure Application Insights.

![dvc_stage_list](DVC_STAGE_LIST.png)

## 1. Contexte et objectifs

### 1.1 Pourquoi analyser la polarité de tweets ?

Les plateformes de réseaux sociaux comme Twitter génèrent un volume considérable de données textuelles. L’analyse de sentiment consiste à déterminer si un tweet exprime une opinion positive, négative ou neutre, ce qui peut avoir de nombreuses applications : suivi de l’image de marque, détection d’opinion politique, satisfaction client, etc.

### 1.2 Les grandes familles de modèles

#### **Méthodes traditionnelles + TF-IDF**

Les modèles comme la Régression Logistique ou le SVM restent souvent efficaces pour des tâches de classification sur des textes courts, d’autant plus lorsque la quantité de données est importante. Le TF-IDF (Term Frequency–Inverse Document Frequency) est un moyen classique de transformer du texte en vecteur numérique, en mettant en avant les termes les plus significatifs.

#### **Réseaux de neurones avancés + Embeddings (Word2Vec, GloVe)**

L’avènement du deep learning a permis de recourir à des représentations de mots plus riches, comme les embeddings GloVe (Global Vectors), qui capturent des similarités sémantiques. En particulier, les réseaux de neurones récurrents (RNN) et leurs variantes, notamment la LSTM bidirectionnelle, sont très populaires pour le traitement du langage naturel.

#### **Transformers + Modèles pré-entraînés (BERT, DistilBERT, etc.)**

Les Transformers ont révolutionné le traitement automatique du langage. Des modèles pré-entraînés tels que BERT, DistilBERT, GPT-2, etc., atteignent des performances élevées grâce à un entraînement sur de grands volumes de textes. DistilBERT est une version allégée de BERT, plus rapide à l’inférence et nécessitant moins de ressources.

## 2. Aperçu du pipeline DVC

![dvc_dag](DVC_DAG.png)


Avant d’entrer dans le détail de chaque approche, il est important de comprendre le fonctionnement général du pipeline. Le fichier dvc.yaml comprend plusieurs étapes regroupées en trois grands blocs :

- **Data Preparation** :
    - Split des données en jeux d’entraînement, de validation et de test.
    - Nettoyage et préparation des tweets pour leur utilisation avec TF-IDF et GloVe.
    - Génération des vecteurs TF-IDF et GloVe.

- **Train Model** :
    - Entraînement du modèle de Régression Logistique sur TF-IDF.
    - Entraînement de la LSTM bidirectionnelle sur embeddings GloVe.
    - Entraînement du modèle DistilBERT (via la librairie ktrain).

- **Test Model** :
    - Tests de performance sur le jeu de test pour chacun des trois modèles.

Enfin, une étape API gère la construction du conteneur Docker et le test de l’API.

### 2.1 La version de données avec DVC

DVC (Data Version Control) permet de :

- Versionner de gros fichiers (corpus de tweets, embeddings, modèles entraînés) sans les stocker directement dans Git.
- Maintenir la reproductibilité en associant chaque jeu de données ou chaque modèle à un hash spécifique.
- Automatiser l’exécution du pipeline complet ou partiel dès qu’un changement est détecté dans le code ou dans les paramètres.

***Le pipeline execute uniquement les parties du pipeline qui sont impacté par un changement***
![dvc_repro](DVC_REPRO.png)

## 3. Modèles développés et comparaison

### 3.1 Modèle sur mesure simple : Régression Logistique + TF-IDF

#### 3.1.1 Présentation

Ce pipeline commence par :
- Extraire les tweets bruts et créer un jeu d’entraînement, validation et test.
- Nettoyer basiquement les tweets : suppression des URL, ponctuation superflue, symboles, etc.
- Appliquer un vecteur TF-IDF sur l’ensemble d’entraînement.

La Régression Logistique est ensuite entraînée pour classifier chaque tweet en polarité positive ou négative.

#### 3.1.2 Avantages et inconvénients

- Avantages :
    - Rapidité d’entraînement et d’inférence.
    - Besoin en ressources faible.
    - Interprétabilité plus simple : on peut inspecter les coefficients associés à chaque mot.

- Inconvénients :
    - Capacité limitée à saisir la sémantique avancée.
    - Sensible à des variations de vocabulaire (e.g., fautes d’orthographe, abréviations non prévues).

#### 3.1.3 Performances

Sur un jeu de tweets standard, la Régression Logistique avec TF-IDF peut atteindre une précision de l’ordre de 75-80 %. Les performances exactes varient selon la qualité du prétraitement et la taille du jeu de données.

***test accurrency 0.76***
![](MLFLOW_RL_TEST_ACC.png)

### 3.2 Modèle sur mesure avancé : BiLSTM + GloVe

#### 3.2.1 Présentation

Dans cette deuxième approche, nous utilisons :

- Des embeddings GloVe : chargés depuis un fichier externe (par exemple glove.twitter.27B.50d.txt).
- Une architecture LSTM bidirectionnelle (BiLSTM) pour gérer les dépendances contextuelles dans le texte, aussi bien dans le sens direct que dans le sens inverse.

#### 3.2.2 Avantages et inconvénients

- Avantages :
    - Les embeddings GloVe capturent des similarités sémantiques.
    - La LSTM bidirectionnelle extrait mieux le contexte que les RNN classiques.

- Inconvénients :
    - Entraînement plus long que la Régression Logistique.
    - Plus complexe à paramétrer (choix du nombre de neurones, etc.).
    - Nécessite un prétraitement supplémentaire (tokenisation alignée avec GloVe, etc.).

#### 3.2.3 Performances typiques

Selon la taille de l’embedding (50, 100, 200 dimensions, etc.) et le nombre d’échantillons, on peut atteindre en général une précision de l’ordre de 80-85 %, voire davantage si le réseau est suffisamment entraîné et que la qualité des données est bonne.

![Mlflow enregistre le model et les artifacts comme les plots](MLFLOW_LSTM_ARTIFACT.png)
## 3.3 Modèle avancé pré-entraîné : DistilBERT via ktrain

#### 3.3.1 Présentation

La troisième approche s’appuie sur DistilBERT, un modèle de type Transformer pré-entraîné sur un vaste corpus de textes. Nous utilisons la bibliothèque ktrain pour :

- Charger DistilBERT et ses poids pré-entraînés.
- Adapter le modèle à notre jeu de tweets pour la classification binaire.

(Commentaire : vous pourriez ajouter ici des extraits de code spécifiques à l’entraînement de DistilBERT avec la librairie ktrain.)

#### 3.3.2 Avantages et inconvénients

- Avantages :
    - Excellentes performances, notamment pour détecter des nuances de langage.
    - Nécessite moins de données annotées pour être performant, grâce au pré-entraînement.
    - Possibilité de couvrir des expressions moins fréquentes ou du vocabulaire spécifique à Twitter.

- Inconvénients :
    - Temps d’inférence plus élevé qu’une simple régression logistique.
    - Besoin de ressources GPU pour un entraînement fluide.
    - Complexité plus grande, ce qui peut rendre l’interprétation du modèle plus délicate.

#### 3.3.3 Performances typiques

DistilBERT peut facilement atteindre et dépasser 85-90 % de précision, selon la qualité du jeu de données et la fine-tuning réalisé.

## 4. La démarche MLOps
### 4.1 Principes MLOps

Le MLOps vise à industrialiser le cycle de vie d’un projet de Machine Learning, en reprenant les principes DevOps appliqués au développement logiciel classique :

- Automatisation : Tests, intégration continue, déploiement continu (CI/CD).
- Versioning : Pour le code, mais aussi pour les données, les modèles, les métriques.
- Collaboration : Des équipes pluridisciplinaires (data scientists, développeurs, ops).
- Monitoring & Observability : Suivi des performances en production, alertes en cas de dérive de données ou de baisse de performance.

## 4.2 Tracking des expérimentations

Dans ce projet, nous pouvons utiliser un outil comme MLflow pour suivre :

- Les hyperparamètres (ex. : taille d’embedding, nombre d’époques, taux d’apprentissage).
- Les métriques (accuracy, loss, etc.).
- Les artefacts (modèles entraînés, courbes de validation).

Chaque modèle produit un fichier mlflow_id.json dans le dossier de sortie, permettant de faire le lien avec la run associée dans MLflow.

![](MLFLOW_ANALYSE.png)

## 4.3 Stockage des modèles et gestion de versions

- DVC se charge de gérer la version des gros fichiers (poids de la LSTM, du logistic model, etc.).
- Git gère la version du code source.
- MLflow conserve les détails de chaque expérience.

Ainsi, il est possible de revenir à n’importe quelle version du pipeline, de reproduire les mêmes résultats et de comprendre quelle configuration a été utilisée.

## 4.4 Tests unitaires et tests d’intégration

Le pipeline inclut également des scripts de test pour chacun des modèles :

- test_model_logistic.py
- test_lstm_bidirectional_with_glove.py
- test_model_distilbert_ktrain.py

Ces scripts se chargent de charger le modèle, d’exécuter des prédictions sur un jeu de test et de sortir les métriques de performance. Intégrer ces tests dans un système de Continuous Integration (par exemple GitHub Actions, GitLab CI/CD ou Azure DevOps) garantit la fiabilité en cas de modification du code.

![](DVC_STAGE_TEST.png)

## 5. Déploiement et suivi en production

### 5.1 Construction et test de l’API

La dernière étape du pipeline (build_and_test_api) exécute un script shell build_and_test_api.sh qui :

- Construit une image Docker à partir du Dockerfile.
- Démarre un conteneur localement.
- Exécute des tests automatisés (test_api.py) pour vérifier que l’API répond correctement.

(Commentaire : vous pouvez insérer ici une capture d’écran de l’API en fonctionnement ou de l’outil de test type Postman / Swagger.)

![](DOCKERFILE.png)

## 5.2 Monitoring en production : Azure Application Insights

Après le déploiement, il est essentiel de surveiller la performance du modèle en production. Azure Application Insights offre :

- Collecte de logs : Temps de réponse, taux d’erreur, etc.
- Suivi des métriques : Nombre de prédictions, utilisation des ressources.
- Alertes personnalisées : Déclenchées si la latence dépasse un certain seuil, ou si le taux d’erreurs augmente.

Pour aller plus loin :

- Envoi des prédictions avec leur “label” final (dans le cas où on connaît la vérité terrain) à une base de données pour mesurer en continu la dérive (concept drift).
- Analyse statistique : Comparaison régulière de la distribution des prédictions en production avec celle en phase de test.
- Mise à jour automatique : Lorsqu’une dérive est détectée, un nouveau cycle d’entraînement peut être déclenché (si un jeu de données étiquetées est disponible).

## 5.3 Approche d’amélioration continue

L’idée est de boucler le cycle MLOps :

- Collecte en continu de nouveaux tweets et de leurs labels réels (lorsqu’ils sont disponibles).
- Analyse des nouvelles données pour comprendre si le modèle reste performant.
- Entraînement itératif : réentraîner périodiquement le modèle avec DVC et MLflow, tout en gardant la traçabilité des versions.
- Déploiement de la nouvelle version du modèle, suivi de tests et de monitoring continu.

Cette démarche garantit que le modèle évolue avec la langue, les nouveaux hashtags ou expressions sur Twitter, et qu’il ne perde pas en pertinence au fil du temps.

## 6. Comparaison finale et conclusion



- Logistic Regression + TF-IDF : Un baseline efficace, rapide à entraîner et déployer. À privilégier pour des projets avec peu de ressources ou pour démarrer rapidement.

- BiLSTM + GloVe : Un compromis équilibré pour capturer des aspects sémantiques plus riches que TF-IDF, sans la complexité totale d’un grand modèle Transformer.

- DistilBERT : Offre la meilleure performance, surtout sur les nuances de langage, au prix d’une infrastructure plus lourde (GPU) et d’une complexité de mise en place plus importante.

Sur le plan MLOps, le pipeline DVC démontre la possibilité de :

- Versionner rapidement jeux de données et modèles.
- Lancer un entraînement reproductible à chaque modification.
- Traquer les expériences grâce à MLflow.
- Déployer une API Docker prête à être testée et monitorée (par exemple via Azure Application Insights).

L’ajout de tests unitaires et de tests d’intégration assure la robustesse du système à chaque mise à jour, tandis que le monitoring permet de suivre la dérive éventuelle en production et de redéclencher des entraînements si nécessaire. À terme, on peut imaginer un système entièrement automatisé (CI/CD) qui récupère de nouveaux tweets en continu, les étiquette, relance l’entraînement et déploie la nouvelle version du modèle, le tout en conservant un historique complet de chaque itération.

7. Ressources supplémentaires

- DVC (Data Version Control) : https://dvc.org/
- MLflow : https://mlflow.org/
- ktrain : https://github.com/amaiya/ktrain
- Azure Application Insights : https://docs.microsoft.com/fr-fr/azure/azure-monitor/app/app-insights-overview

## Mot de la fin

En conclusion, l’utilisation d’une démarche MLOps structurée pour l’analyse de sentiments sur Twitter permet de répondre aux défis du versioning, de la reproductibilité et du déploiement rapide. Que vous optiez pour un modèle simple en Régression Logistique, une BiLSTM enrichie de GloVe ou un modèle Transformer pré-entraîné comme DistilBERT, la clé réside dans l’automatisation, la traçabilité et le monitoring en continu pour maintenir des performances fiables dans le temps.